# 🎵 A2SB: Ses Restorasyonu - Kaggle

**NVIDIA A2SB ile Yüksek Kaliteli Ses Restorasyonu**

Bu notebook ile ses dosyalarınızı AI ile restore edebilirsiniz!

## 🚀 Özellikler
- ✅ 44.1kHz yüksek çözünürlüklü ses restorasyonu
- ✅ Bandwidth extension (yüksek frekans tahmini)
- ✅ Audio inpainting (eksik kısımları doldurma)
- ✅ Kullanıcı dostu Gradio arayüzü

## 📋 Gereksinimler
- 🎯 **GPU**: P100 (16GB) önerilir
- 💾 **RAM**: Minimum 25GB
- ⏱️ **Süre**: 10 saniyelik ses için ~2-3 dakika

## ⚙️ GPU Nasıl Etkinleştirilir?
1. Sağ tarafta **Settings** (Ayarlar) tıklayın
2. **Accelerator** altında **GPU P100** seçin
3. **Save** (Kaydet) tıklayın

## 📚 Kaynaklar
- 📄 [Paper](https://arxiv.org/abs/2501.11311)
- 💻 [GitHub](https://github.com/test4373/diffusion-audio-restoration-colab-Kaggle-.git)
- 🤗 [Models](https://huggingface.co/nvidia/audio_to_audio_schrodinger_bridge)

---

**Kullanım:** Hücreleri sırayla çalıştırın!

## 🔧 Adım 1: CUDA Uyumluluğunu Düzelt (ÖNEMLİ!)

**⚠️ İLK ÖNCE BUNU ÇALIŞTIRIN!**

Kaggle'da PyTorch CUDA 11.8 ama torchvision CUDA 12.4 ile geliyor. Bu uyumsuzluk hatası veriyor.

**Bu hücre:**
1. Uyumsuz paketleri kaldırır
2. CUDA 11.8 uyumlu versiyonları yükler
3. Uyumsuzluğu düzeltir

**⏱️ 2-3 dakika sürer**

In [ ]:
print("="*60)
print("🔧 CUDA UYUMSUZLUĞU DÜZELTİLİYOR")
print("="*60)
print("\n⚠️ Bu Kaggle uyumluluğu için KRİTİK!")
print("⏱️  2-3 dakika sürecek...\n")

# Adım 1: Uyumsuz paketleri kaldır
print("📦 Adım 1/4: Uyumsuz paketler kaldırılıyor...")
!pip uninstall -y torch torchvision torchaudio torchmetrics transformers torchao -q
print("✓ Eski paketler kaldırıldı\n")

# Adım 2: PyTorch CUDA 11.8 ile yükle
print("📦 Adım 2/4: PyTorch CUDA 11.8 yükleniyor...")
!pip install torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 \
    --index-url https://download.pytorch.org/whl/cu118 -q
print("✓ PyTorch yüklendi\n")

# Adım 3: Uyumlu torchmetrics yükle
print("📦 Adım 3/4: Torchmetrics yükleniyor...")
!pip install torchmetrics==1.2.0 --no-deps -q
!pip install numpy packaging -q
print("✓ Torchmetrics yüklendi\n")

# Adım 4: Lightning yükle
print("📦 Adım 4/4: Lightning yükleniyor...")
!pip install lightning==2.1.0 pytorch-lightning==2.1.0 -q
print("✓ Lightning yüklendi\n")

print("="*60)
print("✅ CUDA DÜZELTMESİ TAMAMLANDI!")
print("="*60)
print("\n⚠️ ÖNEMLİ: Şimdi kernel'ı yeniden başlatmalısınız!")
print("\n👉 Session > Restart Session")
print("\nYeniden başlattıktan sonra Adım 2'den devam edin (bu hücreyi atlayın).")
print("="*60)

## ⚠️ KERNEL'I ŞİMDİ YENİDEN BAŞLATIN!

**Yukarıdaki hücreyi çalıştırdıktan sonra:**
1. **Session > Restart Session** gidin
2. Kernel'ın yeniden başlamasını bekleyin
3. Aşağıdaki Adım 2'den devam edin
4. Adım 1'i **TEKRAR ÇALIŞTIRMAYIN**

---

## ✅ Adım 2: Kurulumu Doğrula (Kernel Yeniden Başladıktan Sonra)

**CUDA düzeltmesinin çalıştığını doğrulayalım.**

In [ ]:
import torch
import sys

print("="*60)
print("🔍 KURULUM DOĞRULANIYOR")
print("="*60)

print(f"\n✓ PyTorch: {torch.__version__}")
print(f"✓ PyTorch CUDA: {torch.version.cuda}")
print(f"✓ CUDA Mevcut: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"\n✓ GPU: {torch.cuda.get_device_name(0)}")
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✓ GPU Bellek: {gpu_memory:.1f} GB")
    
    if 'P100' in torch.cuda.get_device_name(0):
        print("\n🎉 Mükemmel! P100 GPU var!")
    
    if gpu_memory < 14:
        print(f"\n⚠️ Uyarı: GPU sadece {gpu_memory:.1f} GB belleğe sahip")
        print("Önerilen: Settings'den P100 GPU (16GB) etkinleştirin")
    else:
        print(f"\n✅ GPU belleği yeterli ({gpu_memory:.1f} GB)")
else:
    print("\n❌ HATA: CUDA mevcut değil!")
    print("Lütfen Settings > Accelerator > GPU P100 etkinleştirin")
    sys.exit(1)

if torch.version.cuda == "11.8":
    print(f"\n✅ CUDA versiyonu doğru: {torch.version.cuda}")
    print("✅ Uyumsuzluk yok - Devam edebilirsiniz!")
else:
    print(f"\n⚠️ Uyarı: PyTorch CUDA versiyonu {torch.version.cuda}")
    print("Beklenen: 11.8")

print("\n" + "="*60)
print("✅ DOĞRULAMA TAMAMLANDI!")
print("="*60)

## 📦 Adım 3: Bağımlılıkları Yükle

**Ses restorasyonu için gerekli kütüphaneleri yükleyelim.**

**⏱️ 3-5 dakika sürer**

In [ ]:
print("📦 Bağımlılıklar yükleniyor...\n")

# Ses işleme kütüphaneleri
print("📥 Ses işleme kütüphaneleri...")
!pip install librosa soundfile scipy einops -q
print("✓ Ses kütüphaneleri yüklendi\n")

# Konfigürasyon araçları
print("📥 Konfigürasyon araçları...")
!pip install jsonargparse[signatures] pyyaml -q
print("✓ Konfigürasyon araçları yüklendi\n")

# Gradio ve yardımcı araçlar
print("📥 Gradio ve yardımcı araçlar...")
!pip install 'huggingface_hub>=0.19.0,<1.0' -q
!pip install gradio==4.44.0 -q
!pip install nest-asyncio tqdm rotary-embedding-torch pyngrok -q
print("✓ Gradio yüklendi\n")

# Opsiyonel: SSR Eval
!pip install ssr-eval -q 2>/dev/null || echo "⚠️ SSR Eval atlandı (opsiyonel)"

print("\n" + "="*60)
print("✅ TÜM BAĞIMLILIKLAR YÜKLENDİ!")
print("="*60)

# Doğrulama
import torch
import lightning
import gradio as gr
import librosa
import nest_asyncio
nest_asyncio.apply()

print(f"\n✓ PyTorch: {torch.__version__}")
print(f"✓ Lightning: {lightning.__version__}")
print(f"✓ Gradio: {gr.__version__}")
print(f"✓ Librosa: {librosa.__version__}")
print("\n🎉 Devam etmeye hazır!")

## 📥 Adım 4: Repository'yi Klonla

**Ses restorasyonu kodunu indirelim.**

In [ ]:
import os

print("📥 Repository klonlanıyor...\n")

os.chdir('/kaggle/working')
!rm -rf diffusion-audio-restoration-colab-Kaggle-
!git clone https://github.com/test4373/diffusion-audio-restoration-colab-Kaggle-.git
os.chdir('diffusion-audio-restoration-colab-Kaggle-')

print(f"\n✅ Repository klonlandı!")
print(f"✓ Dizin: {os.getcwd()}")

## 🤗 Adım 5: Modelleri İndir

**Hugging Face'den iki model checkpoint'i indirelim:**
- One-split (0.0-1.0): ~1.5GB
- Two-split (0.5-1.0): ~1.5GB

**Toplam: ~3GB. 5-10 dakika sürer.**

In [ ]:
import os
from huggingface_hub import snapshot_download

print("🤗 Modeller Hugging Face'den indiriliyor...")
print("⏱️  5-10 dakika sürecek...\n")

model_dir = './pretrained_models'

try:
    snapshot_download(
        repo_id='nvidia/audio_to_audio_schrodinger_bridge',
        local_dir=model_dir,
        local_dir_use_symlinks=False
    )
    print(f"\n✅ Modeller indirildi: {model_dir}")
    print(f"\n📂 Model dosyaları:")
    !ls -lh {model_dir}
    print("\n✅ Model indirme tamamlandı!")
except Exception as e:
    print(f"❌ Model indirme hatası: {e}")
    print("\nSorun Giderme:")
    print("1. İnternet bağlantınızı kontrol edin")
    print("2. Hugging Face'e erişilebildiğini doğrulayın")
    print("3. Bu hücreyi tekrar çalıştırın")

## 🔑 Adım 6: Ngrok Token (Opsiyonel ama Önerilir)

**Ngrok, Gradio arayüzünüz için stabil bir public URL sağlar.**

### Ngrok token nasıl alınır:
1. [ngrok.com](https://ngrok.com/) gidin
2. Ücretsiz hesap oluşturun
3. [Dashboard](https://dashboard.ngrok.com/get-started/your-authtoken) gidin
4. Authtoken'ınızı kopyalayın
5. Aşağıya yapıştırın

**Not:** Ngrok token olmadan Gradio varsayılan paylaşımı kullanır (bu da çalışır).

In [ ]:
import os

# Ngrok token'ınızı buraya yapıştırın (opsiyonel)
NGROK_TOKEN = ""  # Örnek: "2abc123def456ghi789jkl"

if NGROK_TOKEN:
    os.environ['NGROK_TOKEN'] = NGROK_TOKEN
    print("✅ Ngrok token ayarlandı!")
    print("🔗 Gradio stabil bir public URL ile başlatılacak")
else:
    print("⚠️ Ngrok token ayarlanmadı")
    print("📝 Gradio varsayılan paylaşım kullanacak (yine de çalışır)")
    print("\n💡 Stabil URL için ngrok.com'dan token alın")

## 🎨 Adım 7: Gradio Arayüzünü Başlat

### �� Ses restorasyonu için hazır!

**Nasıl kullanılır:**
1. Aşağıdaki hücreyi çalıştırın
2. Çıkan public linke tıklayın
3. Ses dosyası yükleyin
4. Mod seçin:
   - **Bandwidth Extension**: Yüksek frekansları restore et
   - **Inpainting**: Eksik kısımları doldur
5. "🚀 Restore Et" butonuna tıklayın
6. Sonucu dinleyin ve indirin!

**Ayarlar:**
- **Sampling Steps**: 25-100
  - Hızlı: 25-30
  - Dengeli: 50 ⭐
  - En iyi: 75-100
- **Otomatik Cutoff**: Önerilir ⭐

**İşlem Süreleri (P100 GPU):**
- 10s ses, 50 adım: ~2-3 dakika
- 30s ses, 50 adım: ~5-7 dakika
- 60s ses, 50 adım: ~10-15 dakika

In [ ]:
# Gradio arayüzünü başlat
print("🚀 Gradio arayüzü başlatılıyor...\n")
print("⏱️  Lütfen linkin görünmesini bekleyin.\n")
print("="*60)

import os
if os.path.exists('gradio_app_kaggle.py'):
    print("✓ Kaggle-optimize Gradio app kullanılıyor")
    !python gradio_app_kaggle.py --ngrok
else:
    print("⚠️ Varsayılan Gradio app kullanılıyor")
    !python gradio_app.py --share

print("\n" + "="*60)
print("✅ Gradio arayüzü başlatıldı!")
print("Yukarıdaki public linke tıklayarak arayüze erişin.")
print("="*60)

## 💡 İpuçları ve Sorun Giderme

### ❌ CUDA Bellek Hatası

**GPU belleğini temizlemek için bu hücreyi çalıştırın:**

In [ ]:
import gc
import torch

print("🧹 GPU belleği temizleniyor...")

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    
    total = torch.cuda.get_device_properties(0).total_memory / 1e9
    allocated = torch.cuda.memory_allocated(0) / 1e9
    
    print(f"\n✅ GPU belleği temizlendi!")
    print(f"\nBellek Durumu:")
    print(f"  Toplam: {total:.2f} GB")
    print(f"  Kullanılan: {allocated:.2f} GB")
    print(f"  Boş: {total - allocated:.2f} GB")

### 🎯 En İyi Uygulamalar

1. **Küçük başlayın**: 10-20 saniyelik ses ile test edin
2. **Varsayılanları kullanın**: 50 adım, otomatik cutoff
3. **Çıktıları kaydedin**: Hemen indirin
4. **Tek tek işleyin**: Bir seferde bir dosya

### ❌ Yaygın Sorunlar

**CUDA Bellek Hatası:**
- Sampling steps'i 25-30'a düşürün
- Daha kısa ses kullanın
- Yukarıdaki bellek temizleme hücresini çalıştırın

**Import Hataları:**
- Adım 1'i çalıştırıp kernel'ı yeniden başlattığınızdan emin olun
- Adım 3'ü (bağımlılıklar) tekrar çalıştırın

### 📁 Dosya Konumları

- **Giriş**: Gradio üzerinden yükleyin
- **Çıktı**: `/kaggle/working/gradio_outputs/`
- **Modeller**: `/kaggle/working/diffusion-audio-restoration-colab-Kaggle-/pretrained_models/`

### 📖 Kaynaklar

- **Paper**: [arXiv:2501.11311](https://arxiv.org/abs/2501.11311)
- **GitHub**: [Repository](https://github.com/test4373/diffusion-audio-restoration-colab-Kaggle-.git)
- **Models**: [HuggingFace](https://huggingface.co/nvidia/audio_to_audio_schrodinger_bridge)

---

## 🎉 Teşekkürler!

**Alıntı:**
```bibtex
@article{kong2025a2sb,
  title={A2SB: Audio-to-Audio Schrodinger Bridges},
  author={Kong, Zhifeng and Shih, Kevin J and Nie, Weili and Vahdat, Arash and Lee, Sang-gil and Santos, Joao Felipe and Jukic, Ante and Valle, Rafael and Catanzaro, Bryan},
  journal={arXiv preprint arXiv:2501.11311},
  year={2025}
}
```

### ⭐ Projeyi Destekle

- ⭐ [GitHub repository](https://github.com/test4373/diffusion-audio-restoration-colab-Kaggle-.git)'ye yıldız verin
- 🐛 Hata bildirin
- 📢 Paylaşın
- 👍 Bu notebook'u upvote edin

---

**❤️ Ses restorasyonu topluluğu için yapıldı**